### Objective

In this notebook, we develop a global optimization routine to find the design that minimizes the weight while satisfying the geometrical and thermal constraints.

We consider multiple Qs as design specification.

In this notebook, we try to fix the issue of out-of-bound designs.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
from deap import base, creator, tools, algorithms

from two_sources import thermal_distribution_maxT

### 1. Setup constants

In [2]:
df_Q = pd.read_csv('./dataset/Q_test_locations.csv')

In [3]:
Data = (25, 50e-3, 65e-3, 61.4e-3, 106e-3)
Tjmax = 175
c_module, d_module = 61.4e-3, 106e-3

d_min, d_max = 5e-3, 30e-3
b_min, b_max = 73.7e-3, 307e-3
L_min, L_max = 127.2e-3, 530e-3
c_min, c_max = 10e-3, 39e-3
L_duct_min, L_duct_max = 20e-3, 50e-3
n_min, n_max = 10, 50
t_min, t_max = 1e-3, b_max/n_min - 1e-3
Xc1_min, Xc1_max = c_module / 2, b_max - c_module / 2
Xc2_min, Xc2_max = c_module / 2, b_max - c_module / 2
Yc1_min, Yc1_max = d_module / 2, L_max - d_module / 2
Yc2_min, Yc2_max = d_module / 2, L_max - d_module / 2

lb = np.array([5e-3, 73.7e-3, 127.2e-3, 10e-3, 20e-3, 10, 1e-3, 
              c_module / 2, c_module / 2, d_module / 2, d_module / 2])
ub = np.array([30e-3, 307e-3, 530e-3, 39e-3, 50e-3, 50, b_max/n_min - 1e-3,
              b_max - c_module / 2, b_max - c_module / 2, L_max - d_module / 2, 
              L_max - d_module / 2])

#### 2. Problem definition

In [4]:
def objective(x):

    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    # Calculate weight
    density_Al = 2700
    Fan_height = 40e-3
    Fan_Weight = 50.8e-3
    N_fan = np.ceil(b / Fan_height)

    # Weight calculation
    w = density_Al*(b*d*L+n*(c*t*L))+ Fan_Weight*N_fan
    
    return w

In [5]:
def constraint_maxT(x, Q1, Q2, Data, Tmax_threshold):    
    
    # Calculate Tmax
    input = np.hstack((Q1, Q2, x))
    Tmax, _ = thermal_distribution_maxT(input, Data)
    
    # Constraint Tmax to be less than or equal to Tmax_threshold
    if Tmax_threshold >= Tmax:
        return 0
    else:
        return 10000

In [6]:
class EvaluationCounter:
    def __init__(self, func):
        self.func = func
        self.counter = 0

    def __call__(self, *args):
        self.counter += 1
        return self.func(*args)

In [7]:
Tmax_contraint_counter = EvaluationCounter(constraint_maxT)

In [8]:
def constraint_geometric(x, c_module, d_module):

    constraint_list = []

    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    # 1. non_overlap_constraint
    xc_dist = abs(xc1 - xc2)  
    yc_dist = abs(yc1 - yc2)

    if max(xc_dist - c_module, yc_dist - d_module) >= 0:
        constraint_list.append(True)
    else:
        constraint_list.append(False)

    # 2. t upper bound constraint
    if b / n - 1e-3 - t >= 0:
        constraint_list.append(True)
    else:
        constraint_list.append(False)

    # 3. X bounds
    Xc_max = b - c_module / 2
    Xc_min = c_module / 2

    if Xc_max - xc1 >= 0 and xc1 - Xc_min >= 0 and Xc_max - xc2 >= 0 and xc2 - Xc_min >= 0:
        constraint_list.append(True)
    else:
        constraint_list.append(False)

    # 4. Y bounds
    Yc_max = L - d_module / 2
    Yc_min = d_module / 2

    if Yc_max - yc1 >= 0 and yc1 - Yc_min >= 0 and Yc_max - yc2 >= 0 and yc2 - Yc_min >= 0:
        constraint_list.append(True)
    else:
        constraint_list.append(False)

    if np.sum(constraint_list) == len(constraint_list):
        return 0
    else:
        return 10000

In [9]:
def constraint_variable_bounds(x, lb, ub):
    constraint_list = []

    for i, var in enumerate(x):
        if ub[i] - var >= 0 and var - lb[i] >= 0:
            constraint_list.append(True)
        else:
            constraint_list.append(False)
    
    if np.sum(constraint_list) == len(constraint_list):
        return 0
    else:
        return 10000

In [10]:
def make_eval_function(Q1, Q2, Data, c_module, d_module, Tmax_threshold, n_min, n_max, lb, ub):
    def eval_individual(individual):
        # Objective function value
        new_n = int(individual[5])
        individual[5] = max(min(new_n, n_max), n_min)
        objective_value = objective(individual)

        # Penalty for constraint violations
        penalty = 0
            
        try:
            penalty_t = Tmax_contraint_counter(individual, Q1, Q2, Data, Tmax_threshold)
        except Exception as e:
            penalty_t = 10000
            
        penalty += penalty_t
        penalty += constraint_geometric(individual, c_module, d_module)
        penalty += constraint_variable_bounds(individual, lb, ub)

        return objective_value + penalty, 

    return eval_individual

In [11]:
def custom_mutate(individual, low, up, eta, indpb):
    # Clone the individual to avoid altering the original during the iteration
    # Mutate using mutPolynomialBounded for continuous variables
    _, = tools.mutPolynomialBounded(individual, low, up, eta, indpb)
    
    # Copy back the mutated values except for 'n' at index 5
    individual[5] = int(round(individual[5]))

    return individual,

In [12]:
def GA_run(Q1, Q2, lb, ub, num_gen=200):

    start = time.time()
    # Minimization problem. Adjust weights for maximization.
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)
    
    # Initialize DEAP tools for GA
    toolbox = base.Toolbox()
    
    # Individual and population setup
    # Adjust attribute generators according to your problem's variables
    toolbox.register("individual", tools.initCycle, creator.Individual, 
                     (lambda: random.uniform(d_min, d_max),
                      lambda: random.uniform(b_min, b_max),
                      lambda: random.uniform(L_min, L_max),
                      lambda: random.uniform(c_min, c_max),
                      lambda: random.uniform(L_duct_min, L_duct_max),
                      lambda: random.randint(n_min, n_max),
                      lambda: random.uniform(t_min, t_max),
                      lambda: random.uniform(Xc1_min, Xc1_max),
                      lambda: random.uniform(Yc1_min, Yc1_max),
                      lambda: random.uniform(Xc2_min, Xc2_max),
                      lambda: random.uniform(Yc2_min, Yc2_max)), n=1)
    
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    # Register genetic operators
    toolbox.register("evaluate", make_eval_function(Q1, Q2, Data, c_module, d_module, 
                                                    Tjmax, n_min, n_max, lb, ub))
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    # Update mutation registration to use the custom mutation function
    toolbox.register("mutate", tools.mutPolynomialBounded, 
                     low=[d_min, b_min, L_min, c_min, L_duct_min, n_min, t_min, Xc1_min, Yc1_min, Xc2_min, Yc2_min], 
                     up=[d_max, b_max, L_max, c_max, L_duct_max, n_max, t_max, Xc1_max, Yc1_max, Xc2_max, Yc2_max], 
                     eta=1.0, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    # Genetic algorithm parameters
    population_size = 20
    crossover_probability = 0.7
    mutation_probability = 0.5
    number_of_generations = num_gen
    
    # Initialize population
    pop = toolbox.population(n=population_size)
    
    # Statistics accumulator
    stats = tools.Statistics(key=lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    hof_size = 1
    hall_of_fame = tools.HallOfFame(hof_size)
    
    # Evolutionary algorithm execution
    final_pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=crossover_probability, 
                                       mutpb=mutation_probability,
                                       ngen=number_of_generations, stats=stats, 
                                       halloffame=hall_of_fame, verbose=True)

    # Parse results
    best_ind = hall_of_fame[0]
    best_weight = best_ind.fitness.values[0]

    # Runtime calculation
    end = time.time()
    runtime = end - start

    return best_ind, best_weight, runtime, Tmax_contraint_counter.counter

In [13]:
best_designs = []
best_weights = []
runtimes = []
eval_nums = []
best_Tmaxs = []
for i, (Q1, Q2) in enumerate(df_Q.to_numpy()[:2]):
    print(f"Processing {i+1}th iteration with Q1 ({Q1:.2f}) & Q2 ({Q2:2f}) combinations:")
    best_ind, best_weight, runtime, eval_num = GA_run(Q1, Q2, lb, ub, 10)
    best_designs.append(best_ind)
    best_weights.append(best_weight)
    runtimes.append(runtime)
    eval_nums.append(eval_num)

    # Calculate Tmax constraint
    Tmax, _ = thermal_distribution_maxT([Q1, Q2] + list(best_ind), Data)
    best_Tmaxs.append(Tmax)

Processing 1th iteration with Q1 (324.55) & Q2 (273.426620) combinations:


D:\PE-Optimization\new_campaign\two_sources.py:67: RuntimeWarning: invalid value encountered in sqrt
  fRe_fd = 12 / (np.sqrt(EPS) * (1 + EPS) * (1 - 192 * EPS * np.tanh(np.pi / 2 / EPS) / np.pi ** 5))
D:\PE-Optimization\new_campaign\two_sources.py:69: RuntimeWarning: invalid value encountered in sqrt
  fapp = n * visc_air_K * np.sqrt(c * s) * fRe / V


gen	nevals	avg    	min    	max    
0  	20    	23011.9	10005.1	30017.4
1  	10    	20011.5	10005.1	30017.7
2  	18    	20009.5	10005.1	30023.3


D:\PE-Optimization\new_campaign\two_sources.py:104: RuntimeWarning: invalid value encountered in sqrt
  eff_fin = np.tanh(np.sqrt(2 * h * (t + L) / lambda_HS / t / L) * c) / np.sqrt(2 * h * (t + L) / lambda_HS / t / L) / c


3  	19    	16006.3	10003  	30001.8
4  	18    	13506.3	7.3979 	30010.1
5  	18    	16005.9	7.3979 	30011.1
6  	18    	12504.7	7.3979 	20008.8
7  	18    	11005.1	7.46146	30006.5
8  	18    	14505.1	7.8743 	30011.1
9  	17    	10505.3	6.91008	20007.4
10 	16    	10505.1	6.91008	20007.1
Processing 2th iteration with Q1 (332.35) & Q2 (70.869323) combinations:


C:\Users\Shuai\anaconda3\envs\SciML\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\Shuai\anaconda3\envs\SciML\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg    	min   	max    
0  	20    	22012.1	5.2118	30029.7
1  	16    	18506.2	5.2118	30012.9
2  	13    	15006.1	5.2118	30010.3
3  	18    	11507.3	5.2118	30004.3
4  	14    	8506.34	5.2118	30001.8
5  	18    	7505.54	5.2118	20008.5
6  	19    	5005.6 	3.92874	20008.9
7  	16    	9504.95	4.26933	30004.3
8  	16    	7004.82	4.26933	30002.1
9  	15    	4504.93	3.51813	30003.5
10 	16    	4504.51	3.81872	20004.5


In [14]:
df = pd.DataFrame({"Q1": df_Q.to_numpy()[:2, 0],
                  "Q2": df_Q.to_numpy()[:2, 1],
                   "weight": best_weights,
                   "Tmax": best_Tmaxs,
                  "Runtime": runtimes,
                  "Evalnum": eval_nums})
best_designs = np.array(best_designs)
for i, col in enumerate(['d', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2']):
    df[col] = best_designs[:, i]

In [15]:
df

,Q1,Q2,weight,Tmax,Runtime,Evalnum,d,b,L,c,L_duct,n,t,xc1,yc1,xc2,yc2
0,324.548857,273.426620,6.910084,114.494702,24.533144,190,0.027229,0.209934,0.344631,0.036253,0.040388,15.0,0.002542,0.051376,0.215637,0.121387,0.186786
1,332.350489,70.869323,3.518127,139.221930,41.723397,371,0.010798,0.300684,0.306598,0.010640,0.044908,10.0,0.004814,0.227769,0.132680,0.113065,0.165704


#### Verification

In [16]:
def verification_suite(design, Q1, Q2, Data, Tjmax, c_module, d_module):
    
    # Tmax constraint
    check = constraint_maxT(design, Q1, Q2, Data, Tjmax)
    if check >= 0:
        print(f"Tmax verification: True")
    else:
        print(f"Tmax exceeded by {check}")
    
    # Coordinates bounds
    check = x1_upper_bound(design, c_module, d_module)
    if check >= 0:
        print(f"x1 upper bound: True")
    else:
        print(f"x1 upper bound violated: {check}")

    check = x1_lower_bound(design, c_module, d_module)
    if check >= 0:
        print(f"x1 lower bound: True")
    else:
        print(f"x1 lower bound violated: {check}")

    check = x2_upper_bound(design, c_module, d_module)
    if check >= 0:
        print(f"x2 upper bound: True")
    else:
        print(f"x2 upper bound violated: {check}")

    check = x2_lower_bound(design, c_module, d_module)
    if check >= 0:
        print(f"x2 lower bound: True")
    else:
        print(f"x2 lower bound violated: {check}")

    check = y1_upper_bound(design, c_module, d_module)
    if check >= 0:
        print(f"y1 upper bound: True")
    else:
        print(f"y1 upper bound violated: {check}")

    check = y1_lower_bound(design, c_module, d_module)
    if check >= 0:
        print(f"y1 lower bound: True")
    else:
        print(f"y1 lower bound violated: {check}")

    check = y2_upper_bound(design, c_module, d_module)
    if check >= 0:
        print(f"y2 upper bound: True")
    else:
        print(f"y2 upper bound violated: {check}")

    check = y2_lower_bound(design, c_module, d_module)
    if check >= 0:
        print(f"y2 lower bound: True")
    else:
        print(f"y2 lower bound violated: {check}")

    
    # Non-overlapping bounds
    check = non_overlap_constraint(design, c_module, d_module)
    if check >= 0:
        print(f"Non-overlapping True")
    else:
        print(f"Non-overlapping violated: {check}")
    
    # t bounds
    check = t_upper_bound(design)
    if check >= 0:
        print(f"t bound verification True")
    else:
        print(f"t bound violated: {check}")